Set up imports/paths

In [17]:
import numpy as np
import keras
from datetime import datetime
from training_functions import *
data_dir = "C:/Documents/data/"
requests_file = 'request_yamls/training.yaml'
saved_json = 'training.json'

Break up data into training/test

In [4]:
### Only run this cell if there is no training.json file already saved ###
save_training_json(requests_file, data_dir, saved_json)

Extracting activity labels from the HDF5 file
Extracting myo-left data from the HDF5 file
myo-left Data:
  Shape (433918, 8)
Extracting myo-right data from the HDF5 file
myo-right Data:
  Shape (426658, 8)
Extracting tactile-glove-left data from the HDF5 file
tactile-glove-left Data:
  Shape (15946, 32, 32)
Extracting tactile-glove-right data from the HDF5 file
tactile-glove-right Data:
  Shape (15953, 32, 32)
Original Data:
  Shape (2,)
  Sampling rate: 0.00 Hz

Resampled Data:
  Shape (132928,)
  Sampling rate: 50.00 Hz

Extracting myo-left data from the HDF5 file
myo-left Data:
  Shape (433918, 8)

Original Data:
  Shape (433818, 8)
  Sampling rate: 163.19 Hz

Resampled Data:
  Shape (132928, 8)
  Sampling rate: 50.00 Hz

Extracting myo-right data from the HDF5 file
myo-right Data:
  Shape (426658, 8)

Original Data:
  Shape (426558, 8)
  Sampling rate: 160.45 Hz

Resampled Data:
  Shape (132928, 8)
  Sampling rate: 50.00 Hz

Extracting tactile-glove-left data from the HDF5 file
tac

In [2]:
with open(saved_json) as json_file:
    data = ujson.load(json_file)

# Uncomment end of following line to train with both hands, currently only using left-hand data
corresponding_streams = [('myo-left/emg', 'tactile-glove-left/tactile_data')] #,('myo-right/emg', 'tactile-glove-right/tactile_data')]
window_time = 2 # in seconds
train_x, train_y, test_x, test_y = window_data(data, corresponding_streams, window_time)


start windowing
{'Peel a potato'}
100


In [11]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(643, 100, 1)
(643, 100, 1, 1)
(157, 100, 1)
(157, 100, 1, 1)


In [12]:
# replace np.mean with other averaging functions in parse_hdf5_data.py as complexity increases
# currently just averaging all numbers in 32x32 grid and 1x8 emg array
train_x = np.mean(train_x, tuple(range(2, train_x.ndim)), keepdims=True)
train_y = np.mean(train_y, tuple(range(2, train_y.ndim)), keepdims=True)
test_x = np.mean(test_x, tuple(range(2, test_x.ndim)), keepdims=True)
test_y = np.mean(test_y, tuple(range(2, test_y.ndim)), keepdims=True)

In [13]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(643, 100, 1)
(643, 100, 1, 1)
(157, 100, 1)
(157, 100, 1, 1)


Create Model

In [18]:
model = setup_model(train_x)

starting
(None, 100, 1, 1) (None, 100, 1)
layers set up
model created
Model: "prediction_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 1)]          0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                16896     
                                                                 
 dense_2 (Dense)             (None, 100)               6500      
                                                                 
 reshape_2 (Reshape)         (None, 100, 1, 1)         0         
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 100, 1, 1)        2         
 ranspose)                                                       
                                                                 
Total params: 23,398
Trainable params: 23,398


Training/Plotting loop

In [19]:
# https://keras.io/guides/training_with_built_in_methods/
for f in os.listdir('logs/fit/'):
    os.remove('logs/fit/{f}')

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=train_x, 
          y=train_y, 
          batch_size=10,
          epochs=10, 
          validation_data=(test_x, test_y), 
          callbacks=[tensorboard_callback])

Epoch 1/10
65/65 [==============================] - 3s 28ms/step - loss: 0.0323 - accuracy: 0.0000e+00 - val_loss: 0.0078 - val_accuracy: 0.0000e+00
Epoch 2/10
65/65 [==============================] - 2s 24ms/step - loss: 0.0029 - accuracy: 0.0000e+00 - val_loss: 0.0015 - val_accuracy: 0.0000e+00
Epoch 3/10
65/65 [==============================] - 2s 24ms/step - loss: 7.6695e-04 - accuracy: 0.0000e+00 - val_loss: 0.0010 - val_accuracy: 0.0000e+00
Epoch 4/10
65/65 [==============================] - 2s 24ms/step - loss: 6.1750e-04 - accuracy: 0.0000e+00 - val_loss: 9.5840e-04 - val_accuracy: 0.0000e+00
Epoch 5/10
65/65 [==============================] - 1s 23ms/step - loss: 6.0725e-04 - accuracy: 0.0000e+00 - val_loss: 9.5038e-04 - val_accuracy: 0.0000e+00
Epoch 6/10
65/65 [==============================] - 1s 23ms/step - loss: 6.0656e-04 - accuracy: 0.0000e+00 - val_loss: 9.4973e-04 - val_accuracy: 0.0000e+00
Epoch 7/10
65/65 [==============================] - 1s 22ms/step - loss: 6.065

Evaluate trained model

In [20]:
# Can uncomment this once data is split into training, validation, AND testing
# Currently only being split into training and test, with "test" being used to validate in the previous cell

# results = model.evaluate(test_x, test_y, batch_size=128)
# print("test loss, test acc:", results)